# The 4-way Intersection

`pyspect` can set up problems independently from implementations...
At this stage, we only need to import pyspect and select a pyspect language...

In [ ]:
from math import pi

from pyspect import *
from pyspect.langs.ltl import *

TLT.select(ContinuousLTL)

In [ ]:
x = EMPTY
print(x.__require__)
x = AppliedSet('complement', x)
print('x:', x.__require__)
y = EMPTY
t = And(x, y)
print(type(t), t.__require__)

## Environment

Let us first setup the 4-way intersection environment...

In [ ]:
## CONSTANTS ##

# Define origin and size of area, makes it easier to scale up/down later on 
X0, XN = -1.2, 2.4
Y0, YN = -1.2, 2.4

In `pyspect`, we are able to define sets abstractly. They are evaluated [lazily](https://wikipedia.com). For ease-of-use we provide functions for this:
- `BoundedSet`
- `HalfSpaceSet`
- `...`

In [ ]:
## TRAFFIC RULES ##

speedlimit = BoundedSet(v=(0.3, 0.6))

A benefit of using `pyspects` TLTs is that we can mix set definitions and logic statements...
To comply with `center`, a system needs to be member of all provided sets and other defined rules, e.g. `speedlimit`. 

In [ ]:
## INTERSECTION CENTER ##

center = And(
    BoundedSet(x=(X0 + 0.2*XN, X0 + 0.8*XN), y=(Y0 + 0.2*YN, Y0 + 0.8*YN)),
    HalfSpaceSet(normal=[+1, +1], offset=[X0 + 0.25*XN, Y0 + 0.25*YN]),
    HalfSpaceSet(normal=[-1, +1], offset=[X0 + 0.75*XN, Y0 + 0.25*YN]),
    HalfSpaceSet(normal=[-1, -1], offset=[X0 + 0.75*XN, Y0 + 0.75*YN]),
    HalfSpaceSet(normal=[+1, -1], offset=[X0 + 0.25*XN, Y0 + 0.75*YN]),
    speedlimit
)

We see this here as well...

In [ ]:
## ROADS ##

road_e = BoundedSet(y=(Y0 + 0.3*YN, Y0 + 0.5*YN), h=(-pi/5, +pi/5))
road_e = Or(And(road_e, speedlimit), center)

road_w = BoundedSet(y=(Y0 + 0.5*YN, Y0 + 0.7*YN), h=(+pi - pi/5, -pi + pi/5))
road_w = Or(And(road_w, speedlimit), center)

road_n = BoundedSet(x=(X0 + 0.5*XN, X0 + 0.7*XN), h=(+pi/2 - pi/5, +pi/2 + pi/5))
road_n = Or(And(road_n, speedlimit), center)

road_s = BoundedSet(x=(X0 + 0.3*XN, X0 + 0.5*XN), h=(-pi/2 - pi/5, -pi/2 + pi/5))
road_s = Or(And(road_s, speedlimit), center)

Later in this example, we will primarily work with entries and exits...

In [ ]:
## ENTRIES ##

entry_e = And(BoundedSet(x=(X0 + 0.85*XN, X0 + 1.00*XN), y=(Y0 + 0.53*YN, Y0 + 0.67*YN)), road_w)
entry_w = And(BoundedSet(x=(X0 + 0.00*XN, X0 + 0.15*XN), y=(Y0 + 0.33*YN, Y0 + 0.47*YN)), road_e)
entry_n = And(BoundedSet(x=(X0 + 0.33*XN, X0 + 0.47*XN), y=(Y0 + 0.85*YN, Y0 + 1.00*YN)), road_s)
entry_s = And(BoundedSet(x=(X0 + 0.53*XN, X0 + 0.67*XN), y=(Y0 + 0.00*YN, Y0 + 0.15*YN)), road_n)


## EXITS ##

exit_e = And(BoundedSet(x=(X0 + 0.85*XN, X0 + 1.00*XN), y=(Y0 + 0.33*YN, Y0 + 0.47*YN)), road_e)
exit_w = And(BoundedSet(x=(X0 + 0.00*XN, X0 + 0.15*XN), y=(Y0 + 0.53*YN, Y0 + 0.67*YN)), road_w)
exit_n = And(BoundedSet(x=(X0 + 0.53*XN, X0 + 0.67*XN), y=(Y0 + 0.85*YN, Y0 + 1.00*YN)), road_n)
exit_s = And(BoundedSet(x=(X0 + 0.33*XN, X0 + 0.47*XN), y=(Y0 + 0.00*YN, Y0 + 0.15*YN)), road_s)

We can define new functions...

In [ ]:
def FollowTo(constraint, goal_or_constraint, *more):
    if more: # 2nd arg is constraint
        return FollowTo(And(constraint, goal_or_constraint), *more)
    else: # 2nd arg is exit
        return Until(constraint, goal_or_constraint)

Three vehicles in the scenario must follow these rules...

In [ ]:
vehicle_1 = FollowTo(Or(road_e, road_n), exit_n)

vehicle_2 = FollowTo(road_n, Not(vehicle_1), exit_n)

vehicle_3 = FollowTo(road_e, Not(vehicle_1), Not(vehicle_2), exit_e)

## Implementations

Implementations and set representations are completely general. To demonstrate, we define the simplest implementation.
`StrImpl` produces strings that "represent" the sets. This is completetly fine as well...  

In [ ]:
class StrImpl:

    def __init__(self, ndim: int) -> None:
        self.ndim = ndim

    def set_axes_names(self, *args):
        assert len(args) == self.ndim
        self._axes_names = args

    def axis(self, name: str) -> int:
        return self._axes_names.index(name)

    def axis_name(self, i: int) -> str:
        return self._axes_names[i]
    
    def axis_is_periodic(self, i: int) -> bool:
        return False

    def plane_cut(self, normal, offset, axes=None):
        return 'plane{...}'

    def empty(self):
        return 'empty{ }'
    
    def complement(self, vf):
        return f'({vf})^C'
    
    def intersect(self, vf1, vf2):
        return f'({vf1} ∩ {vf2})'

    def union(self, vf1, vf2):
        return f'({vf1} ∪ {vf2})'
    
    def reach(self, target, constraints=None):
        return f'Reach({target}, {constraints})'

    def avoid(self, target, constraints=None):
        return f'Avoid({target}, {constraints})'

In [ ]:
impl = StrImpl(ndim=4)
impl.set_axes_names('x', 'y', 'h', 'v')

out = TLT(vehicle_2).realize(impl)

print(out)

In [ ]:
from hj_reachability.systems import Bicycle4D
from pyspect.impls.hj_reachability import TVHJImpl
from pyspect.plotting.levelsets import *

reach_dynamics = dict(cls=Bicycle4D,
                      min_steer=-5*pi/4,
                      max_steer=+5*pi/4,
                      min_accel=-0.4,
                      max_accel=+0.4)

min_bounds = np.array([   X0,    Y0, -pi, +0.3])
max_bounds = np.array([XN+X0, YN+Y0, +pi, +0.8])
grid_shape = (31, 31, 25, 7)
T = 3

impl = TVHJImpl(reach_dynamics, 
                ('x', 'y', '*h', 'v'),
                min_bounds, max_bounds, grid_shape,
                time_horizon=T)

out = TLT(vehicle_1).realize(impl)

plot3D_levelset(impl.project_onto(out, 0, 1, 2),
                min_bounds=[0, *min_bounds[:2]],
                max_bounds=[T, *max_bounds[:2]])

In [ ]:
from hj_reachability.systems import Bicycle4D
from pyspect.plotting.levelsets import *

reach_dynamics = Bicycle4D(min_steer=-5*pi/4,
                           max_steer=+5*pi/4,
                           min_accel=-0.4,
                           max_accel=+0.4)

min_bounds = np.array([   X0,    Y0, -pi, +0.3])
max_bounds = np.array([XN+X0, YN+Y0, +pi, +0.8])
grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(hj.sets.Box(min_bounds, max_bounds),
                                                               (31, 31, 25, 7),
                                                               periodic_dims=2)

impl = HJImpl(reach_dynamics, grid, 3)
impl.set_axes_names('x', 'y', 'h', 'v')

rect = BoundedSet(x=(X0 + 0.2*XN, X0 + 0.8*XN), y=(Y0 + 0.2*YN, Y0 + 0.8*YN))
wall = HalfSpaceSet(normal=[+1, +1], offset=[X0 + 0.25*XN, Y0 + 0.75*YN])

region = And(rect, wall)
objective = Or(region, Not("obstacles"))
# out = objective.realize(impl, obstacles=...)

# out = TLT.construct(env).realize(impl)
# plot2D_levelset(impl.project_onto(out, 0, 1), 
#                 min_bounds=min_bounds[:2],
#                 max_bounds=max_bounds[:2])